In [2]:
# from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, fbeta_score, roc_auc_score, roc_curve
# from sklearn.model_selection import GridSearchCV, KFold, train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from imblearn.combine import SMOTEENN
# import pandas as pd
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler, OneHotEncoder

# # Load the dataset
# customer_df = pd.read_csv('./Churn_Modelling.csv')

# # Drop unnecessary columns
# customer_df = customer_df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# # Define categorical and numeric features
# categorical_features = ['Geography', 'Gender']
# numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance',
#                     'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']

# print("Numeric Features are", numeric_features)
# print("Categorical Features are", categorical_features)

# # Split the dataset into training and testing sets
# X = customer_df.drop('Exited', axis=1)
# y = customer_df['Exited']

# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# print("The size of the training set is", len(X_train))
# print("The size of the test set is", len(X_test))

# # Function to scale numeric features
# def standard_scaler(df, col_names):
#     features = df[col_names]
#     scaler = StandardScaler().fit(features.values)
#     features = scaler.transform(features.values)
#     df[col_names] = features
#     return df

# # Apply scaling to the numeric features in the training and test sets
# X_train = standard_scaler(X_train, numeric_features)
# X_test = standard_scaler(X_test, numeric_features)

# # Function to perform one-hot encoding on categorical features
# def one_hot_encode(df, col_names):
#     df_encoded = pd.get_dummies(df, columns=col_names, drop_first=True, dtype='float64')
#     return df_encoded

# # Apply one-hot encoding to the categorical features in the training and test sets
# X_train = one_hot_encode(X_train, categorical_features)
# X_test = one_hot_encode(X_test, categorical_features)

# # Verify the shapes of the transformed data
# print("Shape of X_train after encoding and scaling:", X_train.shape)
# print("Shape of X_test after encoding and scaling:", X_test.shape)


Numeric Features are ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
Categorical Features are ['Geography', 'Gender']
The size of the training set is 8000
The size of the test set is 2000
Shape of X_train after encoding and scaling: (8000, 11)
Shape of X_test after encoding and scaling: (2000, 11)


In [9]:
# from imblearn.pipeline import Pipeline, make_pipeline
# # Set up cross-validation
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# # Create the pipeline with SMOTEENN and RandomForestClassifier
# smoteenn_pipeline = make_pipeline(SMOTEENN(random_state=42), 
#                                   RandomForestClassifier(random_state=13))

# # Define parameter grid for GridSearchCV
# params = {
#     'randomforestclassifier__n_estimators': [50, 100, 200],
#     'randomforestclassifier__max_depth': [None, 10, 20, 30],
# }

# # Create a GridSearchCV instance
# smoteenn_rf = GridSearchCV(smoteenn_pipeline, param_grid=params, cv=kf, scoring='recall', return_train_score=True)
# smoteenn_rf.fit(X_train, y_train)

# # Print the best parameters and best score
# print('Best parameters:', smoteenn_rf.best_params_)
# print('Best score:', smoteenn_rf.best_score_)

# # Predict on the test set
# y_pred = smoteenn_rf.best_estimator_.named_steps['randomforestclassifier'].predict(X_test)

# # Calculate the confusion matrix
# cm = confusion_matrix(y_test, y_pred)

# # Calculate various performance metrics
# smoteenn_rf_recall = recall_score(y_test, y_pred)
# smoteenn_rf_precision = precision_score(y_test, y_pred)
# smoteenn_rf_f1 = f1_score(y_test, y_pred)
# smoteenn_rf_f2 = fbeta_score(y_test, y_pred, beta=2)  # Calculate the F2-score
# smoteenn_rf_accuracy = accuracy_score(y_test, y_pred)

# # Print the confusion matrix
# print(cm)

# # Create a DataFrame to store the scores
# ndf = [(smoteenn_rf_recall, smoteenn_rf_precision, smoteenn_rf_f1, smoteenn_rf_f2, smoteenn_rf_accuracy)]
# smoteenn_rf_score = pd.DataFrame(data=ndf, columns=['Recall', 'Precision', 'F1 Score', 'F2 Score', 'Accuracy'])
# smoteenn_rf_score.insert(0, 'Random Forest with', 'SMOTEENN')

# # Display the DataFrame with the scores
# print(smoteenn_rf_score)

# # Calculate AUC-ROC score
# y_pred_proba = smoteenn_rf.best_estimator_.named_steps['randomforestclassifier'].predict_proba(X_test)[:, 1]
# smoteenn_rf_auc = roc_auc_score(y_test, y_pred_proba)

# # Print AUC-ROC score
# print('AUC-ROC Score:', smoteenn_rf_auc)

In [7]:
smoteenn_rf_score

,Random Forest with,Recall,Precision,F1 Score,F2 Score,Accuracy
0,SMOTEENN,0.77887,0.449008,0.569632,0.679092,0.7605


In [10]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, fbeta_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
customer_df = pd.read_csv('./Churn_Modelling.csv')
customer_df = customer_df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Define features and target
X = customer_df.drop('Exited', axis=1)
y = customer_df['Exited']

# Define numeric and categorical features
numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
categorical_features = ['Geography', 'Gender']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Standardize numeric features
def standard_scaler(df, col_names):
    scaler = StandardScaler().fit(df[col_names])
    df[col_names] = scaler.transform(df[col_names])
    return df

X_train = standard_scaler(X_train, numeric_features)
X_test = standard_scaler(X_test, numeric_features)

# One-hot encode categorical features
def one_hot_encode(df, col_names):
    df_encoded = pd.get_dummies(df, columns=col_names, drop_first=True)
    return df_encoded

X_train = one_hot_encode(X_train, categorical_features)
X_test = one_hot_encode(X_test, categorical_features)

# Define function to calculate metrics
def calculate_metrics(y_test, y_pred, y_pred_proba, model_name):
    cm = confusion_matrix(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    f2 = fbeta_score(y_test, y_pred, beta=2)
    accuracy = accuracy_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_pred_proba)

    print(f"Confusion Matrix for {model_name}:")
    print(cm)

    return [model_name, recall, precision, f1, f2, accuracy, auc_roc]

# List to hold all results
results = []

# 1. SVM with Class Weights
class_weights_svm = SVC(class_weight='balanced', probability=True, random_state=42)
class_weights_svm.fit(X_train, y_train)
class_weights_y_pred = class_weights_svm.predict(X_test)
class_weights_y_pred_proba = class_weights_svm.predict_proba(X_test)[:, 1]
results.append(calculate_metrics(y_test, class_weights_y_pred, class_weights_y_pred_proba, 'SVM with Class Weights'))

# 2. SVM with SMOTE + Tomek Links
smote_tomek = SMOTETomek(random_state=42)
X_resampled, y_resampled = smote_tomek.fit_resample(X_train, y_train)
smote_tomek_svm = SVC(probability=True, random_state=42)
smote_tomek_svm.fit(X_resampled, y_resampled)
smote_tomek_y_pred = smote_tomek_svm.predict(X_test)
smote_tomek_y_pred_proba = smote_tomek_svm.predict_proba(X_test)[:, 1]
results.append(calculate_metrics(y_test, smote_tomek_y_pred, smote_tomek_y_pred_proba, 'SVM with SMOTE + Tomek Links'))

# 3. SVM with SMOTE Oversampling
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
smote_svm = SVC(probability=True, random_state=42)
smote_svm.fit(X_resampled, y_resampled)
smote_y_pred = smote_svm.predict(X_test)
smote_y_pred_proba = smote_svm.predict_proba(X_test)[:, 1]
results.append(calculate_metrics(y_test, smote_y_pred, smote_y_pred_proba, 'SVM with SMOTE Oversampling'))

# 4. SVM with Random Oversampling
random_over = RandomOverSampler(random_state=42)
X_resampled, y_resampled = random_over.fit_resample(X_train, y_train)
random_over_svm = SVC(probability=True, random_state=42)
random_over_svm.fit(X_resampled, y_resampled)
random_over_y_pred = random_over_svm.predict(X_test)
random_over_y_pred_proba = random_over_svm.predict_proba(X_test)[:, 1]
results.append(calculate_metrics(y_test, random_over_y_pred, random_over_y_pred_proba, 'SVM with Random Oversampling'))

# 5. SVM with SMOTEENN
smoteenn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train)
smoteenn_svm = SVC(probability=True, random_state=42)
smoteenn_svm.fit(X_resampled, y_resampled)
smoteenn_y_pred = smoteenn_svm.predict(X_test)
smoteenn_y_pred_proba = smoteenn_svm.predict_proba(X_test)[:, 1]
results.append(calculate_metrics(y_test, smoteenn_y_pred, smoteenn_y_pred_proba, 'SVM with SMOTEENN'))

# 6. SVM with No Sampling
no_sampling_svm = SVC(probability=True, random_state=42)
no_sampling_svm.fit(X_train, y_train)
no_sampling_y_pred = no_sampling_svm.predict(X_test)
no_sampling_y_pred_proba = no_sampling_svm.predict_proba(X_test)[:, 1]
results.append(calculate_metrics(y_test, no_sampling_y_pred, no_sampling_y_pred_proba, 'SVM with No Sampling'))

# Create a DataFrame with all results
columns = ['Model', 'Recall', 'Precision', 'F1 Score', 'F2 Score', 'Accuracy', 'AUC-ROC']
results_df = pd.DataFrame(results, columns=columns)

# Display the DataFrame
print(results_df)


Confusion Matrix for SVM with Class Weights:
[[1275  318]
 [ 104  303]]
Confusion Matrix for SVM with SMOTE + Tomek Links:
[[1296  297]
 [ 117  290]]
Confusion Matrix for SVM with SMOTE Oversampling:
[[1297  296]
 [ 117  290]]
Confusion Matrix for SVM with Random Oversampling:
[[1278  315]
 [ 105  302]]
Confusion Matrix for SVM with SMOTEENN:
[[1156  437]
 [  85  322]]
Confusion Matrix for SVM with No Sampling:
[[1563   30]
 [ 242  165]]
                          Model    Recall  Precision  F1 Score  F2 Score  \
0        SVM with Class Weights  0.744472   0.487923  0.589494  0.673633   
1  SVM with SMOTE + Tomek Links  0.712531   0.494037  0.583501  0.654628   
2   SVM with SMOTE Oversampling  0.712531   0.494881  0.584089  0.654923   
3  SVM with Random Oversampling  0.742015   0.489465  0.589844  0.672606   
4             SVM with SMOTEENN  0.791155   0.424242  0.552316  0.674487   
5          SVM with No Sampling  0.405405   0.846154  0.548173  0.452551   

   Accuracy   AUC-ROC  
0

In [11]:
results_df

,Model,Recall,Precision,F1 Score,F2 Score,Accuracy,AUC-ROC
0,SVM with Class Weights,0.744472,0.487923,0.589494,0.673633,0.7890,0.854520
1,SVM with SMOTE + Tomek Links,0.712531,0.494037,0.583501,0.654628,0.7930,0.837114
2,SVM with SMOTE Oversampling,0.712531,0.494881,0.584089,0.654923,0.7935,0.836923
3,SVM with Random Oversampling,0.742015,0.489465,0.589844,0.672606,0.7900,0.852574
4,SVM with SMOTEENN,0.791155,0.424242,0.552316,0.674487,0.7390,0.838472
5,SVM with No Sampling,0.405405,0.846154,0.548173,0.452551,0.8640,0.824076


## Hyperparameter Tuning

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, fbeta_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
customer_df = pd.read_csv('./Churn_Modelling.csv')
customer_df = customer_df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Define features and target
X = customer_df.drop('Exited', axis=1)
y = customer_df['Exited']

# Define numeric and categorical features
numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
categorical_features = ['Geography', 'Gender']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Standardize numeric features
def standard_scaler(df, col_names):
    scaler = StandardScaler().fit(df[col_names])
    df[col_names] = scaler.transform(df[col_names])
    return df

X_train = standard_scaler(X_train, numeric_features)
X_test = standard_scaler(X_test, numeric_features)

# One-hot encode categorical features
def one_hot_encode(df, col_names):
    df_encoded = pd.get_dummies(df, columns=col_names, drop_first=True)
    return df_encoded

X_train = one_hot_encode(X_train, categorical_features)
X_test = one_hot_encode(X_test, categorical_features)

# Define function to calculate metrics
def calculate_metrics(y_test, y_pred, y_pred_proba, model_name):
    cm = confusion_matrix(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    f2 = fbeta_score(y_test, y_pred, beta=2)
    accuracy = accuracy_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_pred_proba)

    print(f"Confusion Matrix for {model_name}:")
    print(cm)

    return [model_name, recall, precision, f1, f2, accuracy, auc_roc]

# List to hold all results
results = []

# Define parameter grid for GridSearchCV
param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__kernel': ['linear', 'rbf', 'poly'],
    'svc__gamma': ['scale', 'auto'],
    'svc__degree': [3, 4, 5],  # Only relevant for 'poly' kernel
    'svc__class_weight': [None, 'balanced']
}

def perform_grid_search(X_train, y_train, model_name, pipeline):
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='recall', n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    print(f'Best parameters for {model_name}: {best_params}')
    print(f'Best recall score for {model_name}: {best_score}')

    # Predict on the test set
    y_pred = grid_search.best_estimator_.predict(X_test)
    y_pred_proba = grid_search.best_estimator_.predict_proba(X_test)[:, 1]

    return calculate_metrics(y_test, y_pred, y_pred_proba, model_name)

# 1. SVM with Class Weights
pipeline_class_weights = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('svc', SVC(class_weight='balanced', probability=True, random_state=42))
])
results.append(perform_grid_search(X_train, y_train, 'SVM with Class Weights', pipeline_class_weights))

# 2. SVM with SMOTE + Tomek Links
pipeline_smote_tomek = Pipeline(steps=[
    ('smote_tomek', SMOTETomek(random_state=42)),
    ('svc', SVC(probability=True, random_state=42))
])
results.append(perform_grid_search(X_train, y_train, 'SVM with SMOTE + Tomek Links', pipeline_smote_tomek))

# 3. SVM with SMOTE Oversampling
pipeline_smote = Pipeline(steps=[
    ('smote', SMOTE(random_state=42)),
    ('svc', SVC(probability=True, random_state=42))
])
results.append(perform_grid_search(X_train, y_train, 'SVM with SMOTE Oversampling', pipeline_smote))

# 4. SVM with Random Oversampling
pipeline_random_over = Pipeline(steps=[
    ('random_over', RandomOverSampler(random_state=42)),
    ('svc', SVC(probability=True, random_state=42))
])
results.append(perform_grid_search(X_train, y_train, 'SVM with Random Oversampling', pipeline_random_over))

# 5. SVM with SMOTEENN
pipeline_smoteenn = Pipeline(steps=[
    ('smoteenn', SMOTEENN(random_state=42)),
    ('svc', SVC(probability=True, random_state=42))
])
results.append(perform_grid_search(X_train, y_train, 'SVM with SMOTEENN', pipeline_smoteenn))

# 6. SVM with No Sampling
pipeline_no_sampling = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('svc', SVC(probability=True, random_state=42))
])
results.append(perform_grid_search(X_train, y_train, 'SVM with No Sampling', pipeline_no_sampling))

# Create a DataFrame with all results
columns = ['Model', 'Recall', 'Precision', 'F1 Score', 'F2 Score', 'Accuracy', 'AUC-ROC']
results_df = pd.DataFrame(results, columns=columns)

# Display the DataFrame
print(results_df)


Fitting 5 folds for each of 144 candidates, totalling 720 fits
